<a href="https://colab.research.google.com/github/agbeyperf/Internship/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **First Practice Section**

# **A. Load Dataset**

In [26]:
import pandas as pd

dataset = pd.read_excel("/content/Book1.xlsx")
dataset.tail()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Standard Type,PubChem Standard Value,IC50,Target Accession(s),Ligand,Target
360,NaN,336948035,85471460,CCOC1=CC=C(C=C1)CC2=CC(=C3C(=C2C)OCO3)[C@H]4[C...,Inactive,NaN,NaN,NaN,IC50,0.00160,1.60,P31639,BDBM228513,Sodium/glucose cotransporter 2
361,NaN,346541718,71078324,CCOC1=CC=C(C=C1)CC2=CC(=C3CCCC3=C2C)[C@H]4[C@@...,Inactive,NaN,NaN,NaN,IC50,0.00120,1.20,P31639,BDBM228480,Sodium/glucose cotransporter 2
362,NaN,346541719,71076966,CCOC1=CC=C(C=C1)CC2=CC(=C3C(=C2Cl)OCCO3)[C@H]4...,Inactive,NaN,NaN,NaN,IC50,0.00071,0.71,P31639,BDBM228481,Sodium/glucose cotransporter 2
363,NaN,346541720,131635528,COC1=CC=C(C=C1)CC2=CC(=C3C(=C2Cl)CCCO3)[C@H]4[...,Inactive,NaN,NaN,NaN,IC50,0.00047,0.47,P31639,BDBM228482,Sodium/glucose cotransporter 2
364,NaN,346541721,71076968,COC1=CC=C(C=C1)CC2=CC(=C3C(=C2Cl)CCS3)[C@H]4[C...,Inactive,NaN,NaN,NaN,IC50,0.00230,2.30,P31639,BDBM228483,Sodium/glucose cotransporter 2


# **B. Data Pre-processing and Separation**

## **Smaller Dataset (3 columns)**

In [27]:
selected_columns = ["PUBCHEM_SID", "PUBCHEM_EXT_DATASOURCE_SMILES", "PUBCHEM_ACTIVITY_OUTCOME"]
concise_dataset = dataset[selected_columns]
concise_dataset.head()

,PUBCHEM_SID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,104153399,C1=CC(=C(C=C1[C@H]2[C@@H]([C@H]([C@@H]([C@H](O...,Active
1,104153400,CCOC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]([C...,Active
2,104153401,CCCOC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]([...,Active
3,104153402,CC(C)OC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]...,Active
4,104153403,CCCCOC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H](...,Active


## **Applying Descriptors**

### **Converting List of Molecules to RDKit Objects**

In [28]:
!pip install rdkit

In [29]:
from rdkit import Chem

smiles_list = concise_dataset["PUBCHEM_EXT_DATASOURCE_SMILES"].tolist()
molecules = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

molecules[:5]

### **Calculating the Molecular Descriptors**

To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors.
  
  1. cLogP (Octanol-water Partition Coefficient)
  2. MW (Molecular Weight)
  3. RB (Number of rotatable bonds)
  4. AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)

#### **LogP, MW and RB**

In [30]:
import numpy as np
from rdkit.Chem import Descriptors

def calculate_descriptors(molecules, verbose=False):

  mol_data = []
  for mol in molecules:
    mol=Chem.MolFromSmiles(mol)
    mol_data.append(mol)

  base_data = np.arange(1, 1)
  i = 0

  for mol in mol_data:
    desc_MolLogP = Descriptors.MolLogP(mol)
    desc_MolWt = Descriptors.MolWt(mol)
    desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)

    row = np.array([desc_MolLogP,
                    desc_MolWt,
                    desc_NumRotatableBonds])

    if i == 0:
      base_data = row
    else:
      base_data = np.vstack([base_data, row])
    i = i + 1

  column_names = ["MolLogP", "MolWt", "NumRotatableBonds"]
  descriptors = pd.DataFrame(data=base_data, columns=column_names)
  return descriptors

In [31]:
df = calculate_descriptors(concise_dataset["PUBCHEM_EXT_DATASOURCE_SMILES"])
df

,MolLogP,MolWt,NumRotatableBonds
0,0.88910,401.246,4.0
1,0.63440,410.854,6.0
2,1.02450,424.881,7.0
3,1.02290,424.881,6.0
4,1.41460,438.908,8.0
...,...,...,...
360,1.22812,432.469,6.0
361,1.98812,428.525,6.0
362,2.94380,482.982,6.0
363,2.95070,468.955,5.0


#### **Aromatic Proportion**

In [32]:
def AromaticAtoms(atom):
  aromatic_atoms = [atom.GetAtomWithIdx(1).GetIsAromatic() for i in range(atom.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i == True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [33]:
# Computing for molecules in the entire dataset
desc_AromaticAtoms = [AromaticAtoms(element) for element in molecules]
desc_AromaticAtoms

[26,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 32,
 26,
 0,
 0,
 0,
 0,
 0,
 0,
 33,
 30,
 30,
 30,
 30,
 31,
 31,
 32,
 0,
 31,
 0,
 0,
 30,
 31,
 0,
 0,
 28,
 0,
 0,
 28,
 29,
 0,
 30,
 31,
 30,
 31,
 31,
 31,
 34,
 34,
 30,
 30,
 29,
 29,
 30,
 30,
 35,
 33,
 30,
 30,
 29,
 29,
 30,
 30,
 30,
 33,
 29,
 0,
 0,
 0,
 0,
 35,
 30,
 30,
 31,
 0,
 0,
 0,
 0,
 32,
 29,
 29,
 0,
 0,
 27,
 32,
 30,
 29,
 25,
 31,
 0,
 32,
 0,
 33,
 32,
 30,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 30,
 30,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 29,
 29,
 0,
 30,
 28,
 0,
 35,
 29,
 29,
 29,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 29,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 29,
 29,
 0,
 0,
 0,
 0,
 0,
 35,
 30,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 32,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 29,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [34]:
# Number of heavy atoms
desc_HeavyAtoms = [element.GetNumHeavyAtoms() for element in molecules]
desc_HeavyAtoms

[26,
 28,
 29,
 29,
 30,
 30,
 31,
 32,
 32,
 32,
 27,
 28,
 32,
 26,
 27,
 28,
 28,
 29,
 29,
 30,
 33,
 30,
 30,
 30,
 30,
 31,
 31,
 32,
 33,
 31,
 34,
 35,
 30,
 31,
 31,
 30,
 28,
 26,
 26,
 28,
 29,
 29,
 30,
 31,
 30,
 31,
 31,
 31,
 34,
 34,
 30,
 30,
 29,
 29,
 30,
 30,
 35,
 33,
 30,
 30,
 29,
 29,
 30,
 30,
 30,
 33,
 29,
 27,
 28,
 29,
 31,
 35,
 30,
 30,
 31,
 33,
 30,
 32,
 33,
 32,
 29,
 29,
 29,
 34,
 27,
 32,
 30,
 29,
 25,
 31,
 28,
 32,
 34,
 33,
 32,
 30,
 31,
 30,
 30,
 31,
 30,
 30,
 30,
 30,
 30,
 31,
 30,
 32,
 32,
 32,
 31,
 31,
 30,
 30,
 30,
 31,
 31,
 31,
 29,
 29,
 30,
 30,
 28,
 28,
 35,
 29,
 29,
 29,
 30,
 30,
 31,
 31,
 31,
 33,
 32,
 31,
 29,
 31,
 29,
 30,
 29,
 28,
 28,
 29,
 29,
 31,
 29,
 28,
 29,
 29,
 29,
 31,
 33,
 34,
 35,
 35,
 30,
 35,
 36,
 33,
 33,
 29,
 29,
 29,
 29,
 31,
 31,
 30,
 30,
 28,
 28,
 33,
 31,
 31,
 32,
 32,
 34,
 30,
 31,
 30,
 32,
 31,
 30,
 31,
 31,
 30,
 31,
 30,
 31,
 32,
 32,
 29,
 32,
 30,
 31,
 32,
 31,
 31,
 31,
 31,


In [35]:
## Computing the Aromatic Proportion (AP) descriptor
desc_AP = [desc_AromaticAtoms[i]/desc_HeavyAtoms[i] for i in range(len(desc_AromaticAtoms))]
desc_AP

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [36]:
## Calculating logS
desc_logS = [Descriptors.MolLogP(element) for element in molecules]
desc_logS = pd.DataFrame(desc_logS, columns=["logS"])
desc_logS

,logS
0,0.88910
1,0.63440
2,1.02450
3,1.02290
4,1.41460
...,...
360,1.22812
361,1.98812
362,2.94380
363,2.95070


In [37]:
## X matric (Combing all computed descriptors into 1 dataframe)

desc_AP_df = pd.DataFrame(desc_AP, columns=['AromaticProportion'])
m = pd.concat([df, desc_AP_df, desc_logS], axis=1)
x = pd.concat([concise_dataset["PUBCHEM_SID"], m], axis=1)
x

,PUBCHEM_SID,MolLogP,MolWt,NumRotatableBonds,AromaticProportion,logS
0,104153399,0.88910,401.246,4.0,1.0,0.88910
1,104153400,0.63440,410.854,6.0,0.0,0.63440
2,104153401,1.02450,424.881,7.0,0.0,1.02450
3,104153402,1.02290,424.881,6.0,0.0,1.02290
4,104153403,1.41460,438.908,8.0,0.0,1.41460
...,...,...,...,...,...,...
360,336948035,1.22812,432.469,6.0,0.0,1.22812
361,346541718,1.98812,428.525,6.0,0.0,1.98812
362,346541719,2.94380,482.982,6.0,0.0,2.94380
363,346541720,2.95070,468.955,5.0,0.0,2.95070


In [38]:
y = concise_dataset["PUBCHEM_ACTIVITY_OUTCOME"]
y

,PUBCHEM_ACTIVITY_OUTCOME
0,Active
1,Active
2,Active
3,Active
4,Active
...,...
360,Inactive
361,Inactive
362,Inactive
363,Inactive


In [39]:
# Checking what other entries are under "y"
unique_values = y.unique()
unique_values

array(['Active', 'Inactive'], dtype=object)

In [40]:
# Converting "Active" and "Inactive" in the y dataset to 1 and 0 using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## **Splitting the Dataset**

In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# **C. Building the Models**

## **1. Linear Regression**

### **Training the model**

In [42]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

LinearRegression()

### **Applying the model to make a prediction**

In [43]:
y_pred_train_lr = regr.predict(x_train)
y_pred_test_lr = regr.predict(x_test)

### **Evaluating model performance**

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy: %.2f" % accuracy_score(y_test, y_pred_test_lr.round()))
print(confusion_matrix(y_test, y_pred_test_lr.round()))
print(classification_report(y_test, y_pred_test_lr.round()))

Accuracy: 1.00
[[26  0]
 [ 0 47]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00        47

    accuracy                           1.00        73
   macro avg       1.00      1.00      1.00        73
weighted avg       1.00      1.00      1.00        73



## **2. Random Forest**

### **Training the model**

In [45]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

### **Applying the model to make predictions**

In [46]:
y_pred_train_rf = rf.predict(x_train)

print("Accuracy: %.2f" % accuracy_score(y_train, y_pred_train_rf.round()))
print("Confusion matrix: ", confusion_matrix(y_train, y_pred_train_rf.round()))

Accuracy: 1.00
Confusion matrix:  [[ 71   0]
 [  0 221]]


In [47]:
y_pred_test_rf = rf.predict(x_test)

# **D. Comparing Model Performances**

In [48]:
from prettytable import PrettyTable

z = PrettyTable()

#z.field_names = ["Model", "Accuracy"]

z.add_column("Model", ["Linear Model", "Random Forest"])
z.add_column("Accuracy", [accuracy_score(y_test, y_pred_test_lr.round()), accuracy_score(y_test, y_pred_test_rf.round())])
z.add_column("Confusion Matrix", [confusion_matrix(y_test, y_pred_test_lr.round()), confusion_matrix(y_test, y_pred_test_rf.round())])
z.add_column("Classification Report", [classification_report(y_test, y_pred_test_lr.round()), classification_report(y_test, y_pred_test_rf.round())])

print(z)


+---------------+----------+------------------+-------------------------------------------------------+
|     Model     | Accuracy | Confusion Matrix |                 Classification Report                 |
+---------------+----------+------------------+-------------------------------------------------------+
|  Linear Model |   1.0    |     [[26  0]     |               precision    recall  f1-score   support |
|               |          |     [ 0 47]]     |                                                       |
|               |          |                  |            0       1.00      1.00      1.00        26 |
|               |          |                  |            1       1.00      1.00      1.00        47 |
|               |          |                  |                                                       |
|               |          |                  |     accuracy                           1.00        73 |
|               |          |                  |    macro avg    